In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import os
from neuralprophet import NeuralProphet
import logging
import urllib.parse as url
import warnings
import numpy as np
from typing import Union, Dict, Optional

warnings.filterwarnings('ignore')

class ForecastingModel:

    def __init__(
            self,
            history_start_date: Union[date, datetime],
            history_end_date: Union[date, datetime],
            forecast_end_date: Union[date, datetime],
            target_data: Dict[str, list],
            exog_data: Dict[str, list],
            target_data_interval: str = 'hour',
            exog_data_interval: str = 'hour'
    ) -> None:
        """
        This method initializes training, validation and testing datasets for training our model.
        :param history_start_date: starting date of our training dataset
        :param history_end_date: ending date of our training dataset / starting date of our testing dataset
        :param forecast_end_date: ending date of our testing dataset
        :param target_data: dictionary contained series names and corresponding column names for target data
        :param exog_data: dictionary contained series names and corresponding column names for exogenous data
        """

        """
        First we fetch data from syspower
        """
        self._dates = {
            'history_start_date': history_start_date,
            'history_end_date': history_end_date,
            'forecast_end_date': forecast_end_date
        }
        self._target_data = self.get_data(data_dict=target_data, interval=target_data_interval)

        temp_date = forecast_end_date + timedelta(days=1)
        exog_data = self.get_data(data_dict=exog_data, interval=exog_data_interval, end_date=temp_date)

        exog_data['ds'] = pd.to_datetime(exog_data['ds'])
        exog_data = exog_data.set_index('ds').resample('H').interpolate()
        exog_data = exog_data.reset_index()
        self._exog_data = exog_data.loc[exog_data['ds'] < temp_date, :]


    @property
    def target_data(self):
        return self._target_data

    @property
    def exog_data(self):
        return self._exog_data

    @property
    def dates(self):
        return self._dates

    def get_data(
            self,
            data_dict: Dict[str, list],
            end_date: Optional[datetime] = None,
            interval: str = 'hour'
    ) -> pd.DataFrame:
        """
        This method loads data from syspower
        :param data_dict: dictionary with series names and corresponding column names
        :param end_date: date where data should ends
        :param interval: string that defines time interval on which data should be fetched
        :return: dataframe with the loaded data
        """
        if end_date:
            request = {
                "series": ','.join(data_dict.get('data_series')),
                'interval': interval,
                'start': self.dates.get('history_start_date').strftime('%Y-%m-%d'),
                'end': end_date.strftime('%Y-%m-%d'),
                'token': '8ycj3jSf2DJZOtX',
                'emptydata': 'yes',
                'currency': '',
                'dateFormat': 'nbno',
                'numberFormat': 'nothousandsdot',
                'fileformat': 'csv',
                'headers': 'no'
            }
        else:
            request = {
                "series": ','.join(data_dict.get('data_series')),
                'interval': interval,
                'start': self.dates.get('history_start_date').strftime('%Y-%m-%d'),
                'end': self.dates.get('forecast_end_date').strftime('%Y-%m-%d'),
                'token': '8ycj3jSf2DJZOtX',
                'emptydata': 'yes',
                'currency': '',
                'dateFormat': 'nbno',
                'numberFormat': 'nothousandsdot',
                'fileformat': 'csv',
                'headers': 'no'
            }

        data_url = f'https://syspower5.skm.no/api/webquery/execute?{url.urlencode(request)}'
        data = pd.read_csv(
            data_url,
            sep=';',
            index_col=0,
            parse_dates=True,
            dayfirst=True,
            header=None,
            names = ['ds'] + data_dict.get('column_names', [])
        ).interpolate().reset_index()
        return data

In [14]:
target_data = {
    'data_series': [
        'CNPNP',
        'CNPSE', 'CNPSE1', 'CNPSE2', 'CNPSE3', 'CNPSE4',
        'CNPNO', 'CNPNO1', 'CNPNO2', 'CNPNO3',
        'CNPDEN', 'CNPDK1', 'CNPDK2',
        'CNPFI'
    ],
    'column_names': [
        'cnp_np',
        'cnp_se', 'cnp_se1', 'cnp_se2', 'cnp_se3', 'cnp_se4',
        'cnp_no', 'cnp_no1', 'cnp_no2', 'cnp_no3',
        'cnp_dk', 'cnp_dk1', 'cnp_dk2',
        'cnp_fi'
    ]
}

exog_data = {
    'data_series': ['SMHITEMPFI_F', 'SMHITEMPSE_F', 'SMHITEMPNO_F', 'SMHITEMPNP_F'],
    'column_names': ['temp_fi', 'temp_se', 'temp_no', 'temp_np']
}

history_start_date = datetime(year=2020, month=12, day=1)
history_end_date = datetime(year=2021, month=12, day=1)
forecast_end_date = datetime(year=2021, month=12, day=10)

forecasting_model = ForecastingModel(
    history_start_date=history_start_date,
    history_end_date=history_end_date,
    forecast_end_date=forecast_end_date,
    target_data=target_data,
    exog_data=exog_data,
    target_data_interval='hour',
    exog_data_interval='day'
)